# Kaggle Tabular Playground Series - Aug 2022
https://www.kaggle.com/competitions/tabular-playground-series-aug-2022<br>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-aug-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-aug-2022/train.csv
/kaggle/input/tabular-playground-series-aug-2022/test.csv


In [2]:
!git clone https://github.com/analokmaus/kuma_utils.git
import sys; sys.path.append("kuma_utils/")
import warnings; warnings.filterwarnings("ignore")

from sklearn.impute import SimpleImputer, KNNImputer
from kuma_utils.preprocessing.imputer import LGBMImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.linear_model import RidgeClassifier

import matplotlib.pyplot as plt

random_seed = 256

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

Cloning into 'kuma_utils'...
remote: Enumerating objects: 915, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 915 (delta 104), reused 102 (delta 96), pack-reused 795
Receiving objects: 100% (915/915), 679.99 KiB | 2.81 MiB/s, done.
Resolving deltas: 100% (592/592), done.


In [3]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2022/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2022/test.csv', index_col='id')

train

,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
id,,,,,,,,,,,,,,,,,,,,,
0,A,80.10,material_7,material_8,9,5,7,8,4,18.040,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,A,84.89,material_7,material_8,9,5,14,3,3,18.213,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,A,82.43,material_7,material_8,9,5,12,1,5,18.057,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,A,101.07,material_7,material_8,9,5,13,2,6,17.295,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,A,188.06,material_7,material_8,9,5,9,2,8,19.346,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26565,E,158.95,material_7,material_6,6,9,6,16,4,16.301,...,NaN,12.177,17.942,10.112,15.795,18.572,16.144,NaN,729.131,0
26566,E,146.02,material_7,material_6,6,9,10,12,8,17.543,...,11.242,14.179,20.564,10.234,14.450,14.322,13.146,16.471,853.924,0
26567,E,115.62,material_7,material_6,6,9,1,10,1,15.670,...,11.407,16.437,17.476,8.668,15.069,16.599,15.590,14.065,750.364,0


In [4]:
# Check for missing values
train.isna().sum().sum()

20273

In [5]:
# Check more detail for missing values
print('Col name', '\t\t Null values', '\t dtype')
print('=================================================')
for col in train.columns:
        print(col, '\t\t',  train[col].isnull().sum(), '\t\t', train[col].dtype)

Col name 		 Null values 	 dtype
product_code 		 0 		 object
loading 		 250 		 float64
attribute_0 		 0 		 object
attribute_1 		 0 		 object
attribute_2 		 0 		 int64
attribute_3 		 0 		 int64
measurement_0 		 0 		 int64
measurement_1 		 0 		 int64
measurement_2 		 0 		 int64
measurement_3 		 381 		 float64
measurement_4 		 538 		 float64
measurement_5 		 676 		 float64
measurement_6 		 796 		 float64
measurement_7 		 937 		 float64
measurement_8 		 1048 		 float64
measurement_9 		 1227 		 float64
measurement_10 		 1300 		 float64
measurement_11 		 1468 		 float64
measurement_12 		 1601 		 float64
measurement_13 		 1774 		 float64
measurement_14 		 1874 		 float64
measurement_15 		 2009 		 float64
measurement_16 		 2110 		 float64
measurement_17 		 2284 		 float64
failure 		 0 		 int64


In [6]:
# Check for duplicated rows
train.duplicated().sum()

0

In [7]:
# Check categorical data in train
for col in ['product_code', 'attribute_0', 'attribute_1']:
    print(col, ': \t', train[col].unique())

product_code : 	 ['A' 'B' 'C' 'D' 'E']
attribute_0 : 	 ['material_7' 'material_5']
attribute_1 : 	 ['material_8' 'material_5' 'material_6']


In [8]:
# Check categorical data in test
for col in ['product_code', 'attribute_0', 'attribute_1']:
    print(col, ': \t', test[col].unique())

product_code : 	 ['F' 'G' 'H' 'I']
attribute_0 : 	 ['material_5' 'material_7']
attribute_1 : 	 ['material_6' 'material_7' 'material_5']


In [9]:
# Get X data
X_train = train.drop(['failure'], axis=1)
X_test = test

# Get y data
y_train = pd.Series(train['failure'])

## One-hot encoding

In [10]:
df = pd.concat((X_train, X_test))
cat_features = ['attribute_0', 'attribute_1']
df = df[cat_features]

# One-Hot Encoding
df = pd.get_dummies(df)

# Merge with train and test, and drop original categorical features
X_train = X_train.join(df).drop(cat_features, axis=1)
X_test = X_test.join(df).drop(cat_features, axis=1)

## Missing values imputation

In [11]:
# Imputing null values
# Create our imputer to replace missing values
#imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = LGBMImputer(n_iter=100)

# Impute data for train and test
features = [f for f in train.columns if f.startswith('measurement') or f=='loading']

for code in X_train.product_code.unique():
    X_train.loc[X_train.product_code==code, features] = imp.fit_transform(X_train.loc[X_train.product_code==code, features])
    
for code in X_test.product_code.unique():
    X_test.loc[X_test.product_code==code, features] = imp.fit_transform(X_test.loc[X_test.product_code==code, features])

# Since product_code is different between train and test, will be dropped
X_train = X_train.drop(['product_code'], axis=1)
X_test = X_test.drop(['product_code'], axis=1)

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [12]:
# PCA Reduction
pca = PCA(n_components='mle', )

X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [13]:
# Scaling data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Model Creation
https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html<br>

In [14]:
# By default model
params = {#'hidden_layer_sizes' : (100,), 
          #'activation' : 'relu', 
          #'solver' : 'adam', 
          'alpha' : 1, 
          #'batch_size' : 'auto', 
          #'learning_rate' : 'constant', 
          #'learning_rate_init' : 0.001, 
          #'power_t' : 0.5, 
          'max_iter' : 2000, 
          #'shuffle' : True, 
          #'tol' : 0.0001, 
          #'warm_start' : False, 
          #'momentum' : 0.9, 
          #'nesterovs_momentum' : True, 
          #'early_stopping' : False, 
          #'validation_fraction' : 0.1, 
          #'beta_1' : 0.9, 
          #'beta_2' : 0.999, 
          #'epsilon' : 1e-08, 
          #'n_iter_no_change' : 10, 
          #'max_fun' : 15000
         }

model = MLPClassifier(**params,
                      random_state=random_seed,
                      #n_jobs=-1,
                      verbose=False,
                      )

## Cross Validation
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html<br>
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html<br>
https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter<br>

In [15]:
cv = StratifiedKFold(n_splits=5)
scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='roc_auc', n_jobs=-1)
#scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc', n_jobs=-1)

scores.mean()

0.5870567391751347

In [16]:
model.get_params()

{'activation': 'relu',
 'alpha': 1,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 2000,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': 256,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

## Optuna Optimization

In [17]:
import optuna

from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

In [18]:
def objective(trial):
       
    hidden_layer_size = trial.suggest_int('hidden_layer_size', 50,1000)
    activation = trial.suggest_categorical('activation', ['logistic', 'relu'])
    alpha = trial.suggest_float('alpha', 1e-3, 0.5, step=1e-3)
    learning_rate_init = trial.suggest_float('learning_rate_init', 1e-4, 0.05, step=1e-4)
    tol = trial.suggest_float('tol', 1e-4, 0.1, step=1e-4)
    
    max_iter = 100000
    #l1_ratio = trial.suggest_float('l1_ratio', 0.0, 1.0, step=1e-2)
        
    params = {'hidden_layer_sizes': (hidden_layer_size,),
              'activation': activation,
              'alpha': alpha,
              'learning_rate_init': learning_rate_init,
              'tol' : tol,
              'max_iter': max_iter,           
             }
    
    model = MLPClassifier(**params,
                          random_state=random_seed,
                          #n_jobs=-1,
                          verbose=False,
                          )
    
    cv = StratifiedKFold(n_splits=5)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='roc_auc', n_jobs=-1)
    #scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1)
        
    return scores.mean()

In [19]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Create Optuna Trial
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.RandomSampler(seed=random_seed))

# Run trials
study.optimize(objective, n_trials = 100)
#study.optimize(objective, timeout = int(3600*5))    # an hour * X

[I 2022-08-18 06:41:34,717] A new study created in memory with name: no-name-14b4f52c-dc5c-484e-bafb-07de692f92c1
[I 2022-08-18 06:41:38,372] Trial 0 finished with value: 0.5885393541378436 and parameters: {'hidden_layer_size': 93, 'activation': 'logistic', 'alpha': 0.043000000000000003, 'learning_rate_init': 0.0013000000000000002, 'tol': 0.0403}. Best is trial 0 with value: 0.5885393541378436.
[I 2022-08-18 06:41:45,462] Trial 1 finished with value: 0.5886052224030872 and parameters: {'hidden_layer_size': 410, 'activation': 'relu', 'alpha': 0.19, 'learning_rate_init': 0.0189, 'tol': 0.0751}. Best is trial 1 with value: 0.5886052224030872.
[I 2022-08-18 06:41:54,118] Trial 2 finished with value: 0.5895031036860482 and parameters: {'hidden_layer_size': 567, 'activation': 'relu', 'alpha': 0.192, 'learning_rate_init': 0.0207, 'tol': 0.04120000000000001}. Best is trial 2 with value: 0.5895031036860482.
[I 2022-08-18 06:41:58,859] Trial 3 finished with value: 0.5704896457084598 and paramete

In [20]:
# Best trial
print('Best trial score:', study.best_trial.value)
study.best_trial.params

Best trial score: 0.5906789182668476


{'hidden_layer_size': 475,
 'activation': 'relu',
 'alpha': 0.062,
 'learning_rate_init': 0.0151,
 'tol': 0.0001}

## Visualization for Hyperparameter Optimization Analysis

In [21]:
plot_optimization_history(study)

In [22]:
plot_param_importances(study)

In [23]:
plot_slice(study)

## Get Best Model

In [24]:
# Create model with best trial parameters
params = {'hidden_layer_sizes': (study.best_trial.params['hidden_layer_size'],),
          'activation': study.best_trial.params['activation'],
          'alpha': study.best_trial.params['alpha'],
          'learning_rate_init': study.best_trial.params['learning_rate_init'],
          'tol' : study.best_trial.params['tol'],
          'max_iter': 10000,
         }

best_model = MLPClassifier(**params,
                          random_state=random_seed,
                          #n_jobs=-1,
                          verbose=False,
                          )

In [25]:
best_model.get_params()

{'activation': 'relu',
 'alpha': 0.062,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (475,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.0151,
 'max_fun': 15000,
 'max_iter': 10000,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': 256,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

## Train Model

In [26]:
# Train best model with all train data
best_model.fit(X_train, y_train)

MLPClassifier(alpha=0.062, hidden_layer_sizes=(475,), learning_rate_init=0.0151,
              max_iter=10000, random_state=256)

## Submission

In [27]:
prediction = best_model.predict(X_test).squeeze()
probabilities = best_model.predict_proba(X_test)[:,1].squeeze()

row_id =  test.index.values
submission = pd.DataFrame({'id' : row_id, 'failure' : probabilities})

submission.to_csv('submission.csv', index=False)

submission.head()

,id,failure
0,26570,0.195582
1,26571,0.153973
2,26572,0.178587
3,26573,0.176954
4,26574,0.297983
